lstm based text generation

In [1]:
!pip install --upgrade pip
!pip install gensim
!pip install gym
!pip install stable-baselines3
!pip install language_tool_python


  Using cached pip-23.1.2-py3-none-any.whl (2.1 MB)


ERROR: To modify pip, please run the following command:
C:\Users\Pablo\anaconda3\python.exe -m pip install --upgrade pip


In [4]:
!pip install pandas numpy tensorflow keras nltk language_tool_python gym stable_baselines3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached stable_baselines3-1.8.0-py3-none-any.whl (174 kB)
  Using cached gym-0.21.0.tar.gz (1.5 MB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, TimeDistributed
from tensorflow.keras.models import Model
import gym
from gym import spaces
from nltk.sentiment import SentimentIntensityAnalyzer
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import language_tool_python
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
import nltk
import numpy as np
from tensorflow.data import Dataset
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.models import Sequential
from numpy import asarray
from numpy import zeros

nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')

# Load the dataset
df = pd.read_csv('aita_clean.csv', low_memory=True)
df = df.dropna()
# Filter the data for each class
not_asshole = df[df['is_asshole'] == 0]
asshole = df[df['is_asshole'] == 1]

# Sample an equal number of instances from each class
sample_size = 25000
not_asshole_sample = not_asshole.sample(sample_size, random_state=42)
asshole_sample = asshole.sample(sample_size, random_state=42)

# Combine the sampled data and shuffle it
df = pd.concat([not_asshole_sample, asshole_sample]).sample(frac=1, random_state=42).reset_index(drop=True)

max_title_len = 20
max_body_len = 200
input_len = max_title_len + max_body_len

all_text = ' '.join(df['title'] + df['body'])
unique_words = set(all_text)
total_words = len(unique_words)

#separator_token = "<SEP>"
#texts = (df['title'] + separator_token + df['body']).tolist()
texts = (df['title'] + ' ' + df['body']).tolist()
tokenizer = Tokenizer(char_level=False)
tokenizer.fit_on_texts(texts)
#tokenizer.word_index[separator_token] = len(tokenizer.word_index) + 1
total_words = len(tokenizer.word_index) + 1

sequences = []
for text in texts:
    token_list = tokenizer.texts_to_sequences([text])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        sequences.append(n_gram_sequence)

max_sequence_len = max([len(seq) for seq in sequences])

sequences = np.array(pad_sequences(sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences, target_sequences = sequences[:, :-1], sequences[:, -1]

input_sequences = pad_sequences(input_sequences, maxlen=input_len, padding='pre')
target_sequences = to_categorical(target_sequences, num_classes=total_words)  # Convert to categorical format


# Load GloVe word embeddings and create an Embeddings Dictionary
embeddings_dictionary = dict()
glove_file = open('a2_glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()
# Create Embedding Matrix having 100 columns
embedding_matrix = zeros((total_words, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
embedding_matrix.shape

#optimisation for slow laptop
#1)sample_size = 100
#2)max_sequence_len = 50  # for laptop sake since its slowwer , its really just fixed


/usr/local/lib/python3.10/dist-packages/google/rpc/__init__.py:20: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google.rpc')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  pkg_resources.declare_namespace(__name__)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2349: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(parent)


ModuleNotFoundError: ignored

In [ ]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 100, weights=[embedding_matrix], input_length=input_len, trainable=False, batch_input_shape=(1, input_len)))
model.add(Bidirectional(LSTM(128, return_sequences=True, stateful=True))) # You can increase the number of LSTM units if you want
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(128, return_sequences=True, stateful=True)))
model.add(TimeDistributed(Dense(total_words, activation='softmax')))


# Compile the model with sparse_categorical_crossentropy
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train the LSTM model for text generation
model.fit(input_sequences, target_sequences, epochs=100, verbose=1, batch_size=1) # Set batch_size=1 to maintain state between batches /should be at least 50 epochs

In [ ]:
model.fit(input_sequences, target_sequences, epochs=1, verbose=1, batch_size=1) # Set batch_size=1 to maintain state between batches /should be at least 50 epochs

In [ ]:
import random

import re

def generate_text(seed_text, next_words, model, max_sequence_len, input_len, tokenizer, total_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=input_len, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)

        # Get the probabilities corresponding to the words in the tokenizer
        filtered_probs = predicted_probs.ravel()[:total_words]

        # Sample a word from the filtered predicted probabilities
        predicted_word_index = random.choices(range(total_words), weights=filtered_probs)[0]
        # Get the corresponding word from the tokenizer
        output_word = ''
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                output_word = word
                break
        seed_text += " " + output_word

    # Split the generated text into title and body
    title_body_split = re.split(r'[.?!]', seed_text, 1)
    title = title_body_split[0]
    body = title_body_split[1] if len(title_body_split) > 1 else ""

    return title, body

In [ ]:
seed_text = "AITA for"
next_words = 50

generated_title, generated_body = generate_text(seed_text, next_words, model, max_sequence_len, input_len, tokenizer, total_words)
print(generated_title)
print(generated_body)


In [ ]:
# # Modify the model architecture for text generation
# input_seq = Input(shape=(None,))
# embedding_layer = Embedding(total_words, 100)
# emb_seq = embedding_layer(input_seq)
# lstm = LSTM(50, return_sequences=True)(emb_seq)
# output = TimeDistributed(Dense(total_words, activation='softmax'))(lstm)
# model = Model(inputs=input_seq, outputs=output)
# model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
# model.fit(train_data, epochs=10, steps_per_epoch=len(X_title_train) // batch_size, validation_data=([X_title_test, X_body_test], y_test))


In [ ]:

# #     def compute_reward(self, sequence):
# #         # Assign weights to each component
# #         w1, w2, w3, w4 = 0.25, 0.25, 0.25, 0.25

# #         # Sentiment analysis reward
# #         sentiment_score = sia.polarity_scores(sequence)
# #         sentiment_reward = 0
# #         if sentiment_score['pos'] > 0 and sentiment_score['neg'] > 0:
# #             sentiment_reward = 1

# #         # Topical coherence reward
# #         coherence_reward = compute_topic_coherence(sequence)

# #         # Post structure reward
# #         post_structure_reward = compute_post_structure_reward(sequence)

# #         # Grammar and fluency reward
# #         grammar_errors = grammar_check(sequence)
# #         grammar_reward = 0
# #         if grammar_errors == 0:
# #             grammar_reward = 1

# #         # Combine the weighted scores to obtain the final reward
# #         reward = (w1 * sentiment_reward +
# #                   w2 * coherence_reward +
# #                   w3 * post_structure_reward +
# #                   w4 * grammar_reward)

# #         return reward

# import gym
# from gym import spaces
# import numpy as np
# from typing import Dict
# from nltk.sentiment import SentimentIntensityAnalyzer
# import language_tool_python

# sia = SentimentIntensityAnalyzer()
# tool = language_tool_python.LanguageTool('en-US')
# vocab_size = 150000
# max_episode_length = 50  # or any suitable value based on your requirements

# def grammar_check(sequence):
#     matches = tool.check(sequence)
#     return len(matches)

# class TextGenerationEnv(gym.Env):
#     def __init__(self, model, tokenizer, max_title_len, max_body_len, max_episode_length):
#         super(TextGenerationEnv, self).__init__()
#         self.max_sequence_len = max_title_len + max_body_len
#         self.model = model
#         self.tokenizer = tokenizer
#         self.max_title_len = max_title_len
#         self.max_body_len = max_body_len
#         self.max_episode_length = max_episode_length

#         # self.action_space = spaces.Dict({
#         #     'title': spaces.MultiDiscrete([vocab_size] * max_title_len),
#         #     'body': spaces.MultiDiscrete([vocab_size] * max_body_len)
#         # })
#         # self.observation_space = spaces.Dict({
#         #     'title': spaces.MultiDiscrete([vocab_size] * max_title_len),
#         #     'body': spaces.MultiDiscrete([vocab_size] * max_body_len)
#         # })
# #         self.action_space = spaces.Dict({
# #             'title': spaces.Discrete(vocab_size),
# #             'body': spaces.Discrete(vocab_size)
# #         })

# #         self.observation_space = spaces.Dict({
# #             'title': spaces.Box(low=0, high=vocab_size, shape=(max_title_len,), dtype=np.int32),
# #             'body': spaces.Box(low=0, high=vocab_size, shape=(max_body_len,), dtype=np.int32)
# #         })


#         self.action_space = spaces.Dict({
#             'type': spaces.Discrete(2),  # 0 for title, 1 for body
#             'token': spaces.Discrete(vocab_size)
#         })

#         self.observation_space = spaces.Dict({
#             'title': spaces.MultiDiscrete([vocab_size] * max_title_len),
#             'body': spaces.MultiDiscrete([vocab_size] * max_body_len)
#         })

#     def reset(self):
#         self.current_title = [0] * self.max_title_len
#         self.current_body = [0] * self.max_body_len
#         self.current_step = 0
#         return {
#             'title': self.current_title,
#             'body': self.current_body
#         }

# #     def step(self, action: Dict[str, np.ndarray]):
# #         for key in ['title', 'body']:
# #             sequence = self.current_title if key == 'title' else self.current_body
# #             max_len = self.max_title_len if key == 'title' else self.max_body_len

# #             sequence[:-max_len] = sequence[max_len:]
# #             sequence[-max_len:] = action[key]

# #             self.current_step += 1



# #     def step(self, action: Dict[str, np.ndarray]):
# #         for key in ['title', 'body']:
# #             sequence = self.current_title if key == 'title' else self.current_body
# #             max_len = self.max_title_len if key == 'title' else self.max_body_len

# #             sequence[:-1] = sequence[1:]
# #             sequence[-1] = action[key]

# #         self.current_step += 1
#     def step(self, action: Dict[str, np.ndarray]):
#         sequence_type = action['type']
#         token = action['token']
#         sequence = self.current_title if sequence_type == 0 else self.current_body
#         max_len = self.max_title_len if sequence_type == 0 else self.max_body_len

#         sequence[:-1] = sequence[1:]
#         sequence[-1] = token

#         self.current_step += 1

#         # Compute the reward based on the generated text (use a custom reward function)
#         title_reward = self.compute_title_reward(self.current_title)
#         body_reward = self.compute_body_reward(self.current_body)

#         # Combine the weighted scores to obtain the final reward
#         w_title, w_body = 0.5, 0.5
#         reward = (w_title * title_reward + w_body * body_reward)
#         reward /= (w_title + w_body)

#         done = self.current_step >= self.max_episode_length

#         return {
#             'title': self.current_title,
#             'body': self.current_body
#         }, reward, done, {}
#  # body = tokenizer.sequences_to_texts([body_sequence])[0]
#     # # Sentiment analysis reward
#     # sentiment_score = sia.polarity_scores(body)
#     # reward = 0
#     # if sentiment_score['pos'] > 0 and sentiment_score['neg'] > 0:
#     #     reward += 1



# def compute_title_reward(self, title_sequence):
#     title = self.tokenizer.sequences_to_texts([title_sequence])[0]
#     title_reward = 0
#     title_keywords = ['aita', 'wibta', 'am i the']
#     title_present = any(keyword.lower() in title.lower() for keyword in title_keywords)
#     if title_present:
#         title_reward += 1

#     grammar_errors = grammar_check(title)
#     grammar_reward = 0
#     if grammar_errors == 0:
#         grammar_reward += 1

#     # Combine the weighted scores to obtain the final reward
#     w_title, w_grammar = 0.5, 0.5
#     reward = (w_title * title_reward + w_grammar * grammar_reward)
#     reward /= (w_title + w_grammar)

#     return reward

# def compute_body_reward(self, body_sequence):
#     body = self.tokenizer.sequences_to_texts([body_sequence])[0]

#     # Sentiment analysis reward
#     sentiment_score = sia.polarity_scores(body)
#     sentiment_reward = 1 if abs(sentiment_score['compound']) >= 0.75 else 0

#     # Topical coherence reward
#     coherence_reward = self.compute_topic_coherence(body)

#     # Post structure reward
#     situation_keywords = ['situation', 'happened', 'issue']
#     action_keywords = ['action', 'did', 'took']
#     justifiable_keywords = ['justifiable', 'wrong', 'right']
#     situation_present = any(keyword.lower() in body.lower() for keyword in situation_keywords)
#     action_present = any(keyword.lower() in body.lower() for keyword in action_keywords)
#     justifiable_present = any(keyword.lower() in body.lower() for keyword in justifiable_keywords)
#     structure_reward = sum([situation_present, action_present, justifiable_present])

#     # Grammar and fluency reward
#     grammar_errors = grammar_check(body)
#     grammar_reward = 1 if grammar_errors == 0 else 0

#     # Combine the weighted scores to obtain the final reward
#     w_sentiment, w_coherence, w_structure, w_grammar = 1, 1, 3, 1
#     reward = (w_sentiment * sentiment_reward +
#               w_coherence * coherence_reward +
#               w_structure * structure_reward +
#               w_grammar * grammar_reward)
#     reward /= (w_sentiment + w_coherence + w_structure + w_grammar)

#     return reward

#    def compute_topic_coherence(self):
#     # Preprocess the title and body
#     title = self.tokenizer.sequences_to_texts([self.current_title])[0]
#     body = self.tokenizer.sequences_to_texts([self.current_body])[0]
#     tokenized_title = preprocess_text_lda(title)
#     tokenized_body = preprocess_text_lda(body)

#     # Combine the tokenized title and body
#     tokenized_sequence = tokenized_title + tokenized_body

#     # Create a bag-of-words representation
#     bow_sequence = dictionary.doc2bow(tokenized_sequence)

#     # Get topic distribution for the generated sequence
#     topic_dist = lda_model.get_document_topics(bow_sequence)

#     # Compute coherence based on the highest probability topic
#     coherence = max([prob for _, prob in topic_dist])

#     return coherence


In [ ]:
import gym
from gym import spaces
import numpy as np
from typing import Dict
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import language_tool_python

from gensim.models import CoherenceModel
#sentiment analysis
sia = SentimentIntensityAnalyzer()
tool = language_tool_python.LanguageTool('en-US')
#max_episode_length = 50  #max_episode_length is a term commonly used in reinforcement learning, referring to the maximum number of steps (or actions) allowed in each episode of the learning environment.
max_episode_length = input_len

#similarity between title and body
def compute_cosine_similarity(title, body):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([title, body])
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
    return similarity

def grammar_check(sequence):
    matches = tool.check(sequence)
    return len(matches)

def preprocess_for_ppo(generated_text):
    tokenized_text = preprocess_text_lda(generated_text)
    return tokenized_text



class TextGenerationEnv(gym.Env):
    def __init__(self, model, tokenizer, max_title_len, max_body_len, max_episode_length, preprocess_for_ppo):
        super(TextGenerationEnv, self).__init__()
        self.preprocess_fn = preprocess_for_ppo
        self.max_sequence_len = max_title_len + max_body_len
        self.model = model
        self.tokenizer = tokenizer
        self.max_title_len = max_title_len
        self.max_body_len = max_body_len
        self.max_episode_length = max_episode_length
        self.action_space = spaces.Discrete(total_words)
        self.observation_space = spaces.Box(low=0, high=total_words - 1, shape=(max_title_len + max_body_len,), dtype=np.int32)

    def reset(self):
        self.current_text = self.generate_initial_state()
        self.current_title = self.current_text[:self.max_title_len]
        self.current_step = 0
        #print("Reset: current_text shape", self.current_text.shape)  # Add this line
        print("Reset: current_text length", len(self.current_text))
        return self.current_text

    def generate_initial_state(self):
        initial_title = np.random.randint(1, total_words, self.max_title_len - 1)
        initial_body = np.random.randint(1, total_words, self.max_body_len)
        initial_text = np.concatenate((initial_title, initial_body))
        return initial_text


    def step(self, action: np.ndarray):
        token = action
        self.current_text[:-1] = self.current_text[1:]
        self.current_text[-1] = token

        self.current_step += 1
        generated_text = self.tokenizer.sequences_to_texts([self.current_text])[0]

        # Split the generated text into title and body
        title_body_split = re.split(r'[.?!]', generated_text, 1)
        title = title_body_split[0]
        body = title_body_split[1] if len(title_body_split) > 1 else ""

        # Compute the reward using the compute_reward function
        reward = self.compute_reward(title, body)

        input_sequences = np.expand_dims(self.current_text[:-1], axis=0)
        target_sequences = np.expand_dims(self.current_text[1:], axis=0)
        self.model.fit(input_sequences, target_sequences, epochs=1, verbose=0, batch_size=1)

        done = self.current_step >= self.max_episode_length

        return self.current_text, reward, done, {}









#     def compute_title_reward(self, title_sequence):
#         title = self.tokenizer.sequences_to_texts([title_sequence])[0]
#         title_reward = 0
#         title_keywords = ['aita', 'wibta', 'am i the']
#         title_present = any(keyword.lower() in title.lower() for keyword in title_keywords)
#         if title_present:
#             title_reward += 1

#         grammar_errors = grammar_check(title)
#         grammar_reward = 0
#         if grammar_errors == 0:
#             grammar_reward += 1

#         # Combine the weighted scores to obtain the final reward
#         w_title, w_grammar = 0.5, 0.5
#         reward = (w_title * title_reward + w_grammar * grammar_reward)
#         reward /= (w_title + w_grammar)

#         return reward

    def compute_title_reward(self, preprocessed_title, preprocessed_body):
        title_reward = 0
        title_keywords = ['aita', 'wibta', 'am i the']
        title_present = any(keyword in preprocessed_title for keyword in title_keywords)
        if title_present:
            title_reward += 1

        grammar_errors = grammar_check(' '.join(preprocessed_title))
        grammar_reward = 0
        if grammar_errors == 0:
            grammar_reward += 1

        # Cosine similarity between title and body
        title_body_similarity = compute_cosine_similarity(' '.join(preprocessed_title), ' '.join(preprocessed_body))

        # Combine the weighted scores to obtain the final reward
        w_title, w_grammar, w_similarity = 0.4, 0.4, 0.2
        reward = compute_weighted_score([w_title, w_grammar, w_similarity], [title_reward, grammar_reward, title_body_similarity])

        return reward


    def compute_body_reward(self, preprocessed_title, preprocessed_body):
        body_reward = 0
        body = ' '.join(preprocessed_body)

        # Sentiment analysis reward
        sentiment_score = sia.polarity_scores(body)
        sentiment_reward = sentiment_score['compound']

        # Topical coherence reward
        coherence_reward = self.compute_topic_coherence(body)

        # Post structure reward
        situation_keywords = ['situation', 'happened', 'issue']
        action_keywords = ['action', 'did', 'took']
        justifiable_keywords = ['justifiable', 'wrong', 'right']
        situation_present = any(keyword in preprocessed_body for keyword in situation_keywords)
        action_present = any(keyword in preprocessed_body for keyword in action_keywords)
        justifiable_present = any(keyword in preprocessed_body for keyword in justifiable_keywords)
        w_situation, w_action, w_justifiable = 2, 1, 1
        structure_reward = (w_situation * situation_present +
                            w_action * action_present +
                            w_justifiable * justifiable_present)
        structure_reward /= (w_situation + w_action + w_justifiable)

        # Grammar and fluency reward
        grammar_errors = grammar_check(body)
        grammar_reward = max(0, 1 - grammar_errors / len(preprocessed_body))

        # Cosine similarity between title and body
        title_body_similarity = compute_cosine_similarity(' '.join(preprocessed_title), body)

        # Combine the weighted scores to obtain the final reward
        w_sentiment, w_coherence, w_structure, w_grammar, w_similarity = 1, 1, 3, 1, 1
        reward = compute_weighted_score([w_sentiment, w_coherence, w_structure, w_grammar, w_similarity], [sentiment_reward, coherence_reward, structure_reward, grammar_reward, title_body_similarity])

        return reward


    def compute_topic_coherence(self, body):
        # Preprocess the title and body
        title = self.tokenizer.sequences_to_texts([self.current_title])[0]
        tokenized_title = preprocess_text_lda(title)
        tokenized_body = preprocess_text_lda(body)

        # Combine the tokenized title and body
        tokenized_sequence = tokenized_title + tokenized_body

        # Create a bag-of-words representation
        bow_sequence = dictionary.doc2bow(tokenized_sequence)

        # Get topic distribution for the generated sequence
        topic_dist = lda_model.get_document_topics(bow_sequence)

        # Get the most probable topic
        most_probable_topic = max(topic_dist, key=lambda x: x[1])[0]

        # Compute coherence using Gensim's CoherenceModel
        coherence_model_lda = CoherenceModel(model=lda_model, texts=[tokenized_sequence], dictionary=dictionary, coherence='c_v')
        coherence = coherence_model_lda.get_coherence_per_topic()[most_probable_topic]

        return coherence


    def compute_reward(self, title, body):
        # Preprocess the title and body
        preprocessed_title = preprocess_text_lda(title)
        preprocessed_body = preprocess_text_lda(body)

        # Compute rewards for the title and body
        title_reward = self.compute_title_reward(preprocessed_title, preprocessed_body)
        body_reward = self.compute_body_reward(preprocessed_title, preprocessed_body)

        # Combine the title and body rewards with appropriate weights
        w_title, w_body = 0.5, 0.5
        reward = (w_title * title_reward + w_body * body_reward)
        reward /= (w_title + w_body)

        return reward



In [ ]:
#     def compute_reward(self, sequence):
#         reward = 0
#         # Compute cosine similarity between consecutive words in the sequence
#         for i in range(len(sequence) - 1):
#             word1 = self.tokenizer.index_word.get(sequence[i], None)
#             word2 = self.tokenizer.index_word.get(sequence[i+1], None)

#             if word1 is not None and word2 is not None:
#                 embedding1 = embeddings_dictionary.get(word1, None)
#                 embedding2 = embeddings_dictionary.get(word2, None)

#                 if embedding1 is not None and embedding2 is not None:
#                     cosine_similarity = dot(embedding1, embedding2) / (norm(embedding1) * norm(embedding2))
#                     reward += cosine_similarity

#         return reward

This compute_reward function computes the reward based on the cosine similarity between consecutive words in the generated sequence. It sums up the cosine similarities for all consecutive word pairs, encouraging the model to generate text with similar words in close proximity. Note that this is just an example reward function, and you may need to experiment with different reward functions depending on your specific requirements.


The cosine similarity-based reward function I provided is more focused on generating sentences that make sense by considering the similarity between consecutive words. For generating AITA posts, you might want to use a more complex reward function that takes into account additional factors.

Here are a few ideas for a custom reward function tailored to AITA posts:

Sentiment analysis: AITA posts often involve a mix of emotions. You can use a pre-trained sentiment analysis model (such as VADER or TextBlob) to analyze the sentiment of the generated text and reward the model based on the presence of a mix of emotions.

Topical coherence: You can use topic modeling techniques (such as LDA) to ensure that the generated post is coherent and focused on a specific topic. Reward the model based on the coherence score.

Post structure: AITA posts usually follow a particular structure: they present a situation, describe the actions taken, and ask if the actions were justifiable. You can create a custom reward function that evaluates if the generated text follows this structure.

Grammar and fluency: Reward the model based on the grammatical correctness and fluency of the generated text. You can use tools like the LanguageTool library for grammar checking.

In [ ]:

#Topical coherence

from gensim.models import LdaModel
from gensim.corpora import Dictionary
from nltk.tokenize import word_tokenize

# Preprocess the text
def remove_tags(text):
    TAG_RE = re.compile(r'<[^>]+>')
    return TAG_RE.sub('', text)

def preprocess_text_lda(text):
    corpus = []
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')

    # Lowercase
    text = text.lower()

    # Remove HTML tags
    text = remove_tags(text)

    # Remove punctuations and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Remove single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)

    # Remove stopwords
    text = pattern.sub('', text)

    # Tokenize the text
    tokenized_text = word_tokenize(text)

    return tokenized_text


# Create a list of tokenized titles and bodies
tokenized_titles = [preprocess_text_lda(text) for text in df['title']]
tokenized_bodies = [preprocess_text_lda(text) for text in df['body']]

# Combine the tokenized titles and bodies into a single list of documents
documents = [title + body for title, body in zip(tokenized_titles, tokenized_bodies)]


# Create a dictionary representation of the documents
dictionary = Dictionary(documents)

# Filter out words that occur less than 20 documents, or more than 50% of the documents
dictionary.filter_extremes(no_below=20, no_above=0.5)

# Create a bag-of-words representation of the documents
corpus = [dictionary.doc2bow(doc) for doc in documents]


# Train the LDA model with increased passes and iterations
num_topics = 10  # You can choose the number of topics based on your dataset
passes = 20  # Increase the number of passes
iterations = 200  # Increase the number of iterations
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, random_state=42, passes=passes, iterations=iterations)




#Your custom reward function looks good for checking the post structure based on the presence of specific keywords. However, if you're using a combined dataset of title and body,
#you might want to modify the function to ensure the generated text has a distinct title and body that follow the structure of AITA posts.







In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback

# Create a training environment
train_env = TextGenerationEnv(model, tokenizer, max_title_len, max_body_len, max_episode_length, preprocess_for_ppo)
train_env = DummyVecEnv([lambda: train_env])

# Train the PPO agent
ppo_agent = PPO("MlpPolicy", train_env, verbose=1)

# Create an evaluation environment
eval_env = TextGenerationEnv(model, tokenizer, max_title_len, max_body_len, max_episode_length, preprocess_for_ppo)
eval_env = DummyVecEnv([lambda: eval_env])

# Use the same PPO agent for evaluation
agent = ppo_agent

# Create the SaveBestModelCallback instance
callback = SaveBestModelCallback(save_path, eval_env, check_freq=1000)

# Train the agent with the callback
agent.learn(total_timesteps=timesteps, callback=callback)

eval_callback = EvalCallback(eval_env, best_model_save_path='./models/',
                             log_path='./logs/', eval_freq=10000,
                             deterministic=True, render=False,
                             save_best=True)

ppo_agent.learn(total_timesteps=50000, callback=eval_callback)

# Load the best model
best_agent = PPO.load(save_path)

# Generate text using the trained agent
obs = eval_env.reset()
done = False
generated_text = []

while not done:
    action, _states = best_agent.predict(obs)
    obs, reward, done, info = env.step(action)
    generated_text.append(tokenizer.sequences_to_texts([obs])[0])

# Join the generated text and split it into title and body using the separator token
generated_text = " ".join(generated_text)
generated_title, generated_body = generated_text.split(separator_token)

print("Generated title:", generated_title)
print("Generated body:", generated_body)

# Evaluate the agent
total_rewards = []
for _ in range(35):  # You can adjust the number of evaluation episodes
    obs = eval_env.reset()
    done = False
    episode_reward = 0
    while not done:
        action, _ = best_agent.predict(obs, deterministic=True)
        obs, reward, done, info = eval_env.step(action)
        episode_reward += reward
    total_rewards.append(episode_reward)

avg_reward = np.mean(total_rewards)
print(f"Average reward for the best model: {avg_reward}")


In [ ]:
from stable_baselines3.common.callbacks import BaseCallback

class SaveBestModelCallback(BaseCallback):
    def __init__(self, save_path, eval_env, check_freq, verbose=0):
        super(SaveBestModelCallback, self).__init__(verbose)
        self.save_path = save_path
        self.eval_env = eval_env
        self.check_freq = check_freq
        self.best_reward = -float("inf")

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:
            # Evaluate the current model
            total_rewards = []
            for _ in range(5):  # You can adjust the number of evaluation episodes
                obs = self.eval_env.reset()
                done = False
                episode_reward = 0
                while not done:
                    action, _ = self.model.predict(obs, deterministic=True)
                    obs, reward, done, _ = self.eval_env.step(action)
                    episode_reward += reward
                total_rewards.append(episode_reward)

            avg_reward = np.mean(total_rewards)
            print(f"Average reward after {self.n_calls} steps: {avg_reward}")  # Add this line to print the average reward
            if avg_reward > self.best_reward:
                self.best_reward = avg_reward
                self.model.save(self.save_path)
                if self.verbose:
                    print(f"New best model with reward {avg_reward}, model saved.")

        return True


save_path = "best_model"
eval_env = TextGenerationEnv(model, tokenizer, max_sequence_len, max_episode_length)
eval_env = DummyVecEnv([lambda: eval_env])
callback = SaveBestModelCallback(save_path, eval_env, check_freq=5000)


for i, sample in enumerate(generated_samples):
    print(f"Sample {i + 1}:")
    print(sample)
    print("-" * 40)


In [ ]:
#use gpt 2
# from transformers import GPT2LMHeadModel

# teacher_model = GPT2LMHeadModel.from_pretrained("gpt2")

# from transformers import GPT2Config

# student_config = GPT2Config.from_pretrained("gpt2", n_layer=6, n_embd=768 // 2, n_head=12, n_positions=1024)
# student_model = GPT2LMHeadModel(student_config)

# import torch

# def distillation_loss(student_logits, teacher_logits, temperature=2.0):
#     student_probs = torch.nn.functional.log_softmax(student_logits / temperature, dim=-1)
#     teacher_probs = torch.nn.functional.softmax(teacher_logits / temperature, dim=-1)
#     return torch.nn.functional.kl_div(student_probs, teacher_probs, reduction="batchmean")
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(
#     output_dir="./results",
#     overwrite_output_dir=True,
#     num_train_epochs=3,
#     per_device_train_batch_size=4,
#     save_steps=10_000,
#     save_total_limit=2,
# )

# def compute_loss(model, inputs, teacher_model):
#     student_logits = model(**inputs).logits
#     with torch.no_grad():
#         teacher_logits = teacher_model(**inputs).logits
#     return distillation_loss(student_logits, teacher_logits)

# trainer = Trainer(
#     model=student_model,
#     args=training_args,
#     compute_loss=compute_loss,
#     train_dataset=tokenized_dataset,
#     tokenizer=tokenizer,
# )

# trainer.train()

In [ ]:

from transformers import pipeline

generator = pipeline("text-generation", model="distilled_gpt2", tokenizer=tokenizer)

generated_text = generator("AITA for", max_length=50, do_sample=True, temperature=0.8)
print(generated_text[0]['generated_text'])
